# LangChain이란?
* Language : Large Language Model의 Language
* Chain: 모듈들을 체인으로 연결하여 다른 툴의 출력을 다음 툴의 입력으로 연결하는 방식
  * 예: 검색 -> OpenAI(LLM) -> 계산
* LLM(Large Language Model)을 이용해서 뭔가 만들때 쓸수 있는 툴/라이브러리 모음.
* 파이썬과 자바스크립트 라이브러리를 제공.
* 검색, 메모리, 계산, 요약, 색인, Agent(에이전트) 등의 기능을 제공
* 모듈화로 쉽게 부품을 갈아끼울수 있음
* LLM은 기본 OpenAI (text-davinci-003) 이용

---
참고 자료:
* 파이썬 문서: https://python.langchain.com/en/latest/index.html
* 개념 문서: https://docs.langchain.com/docs/
* 소스: https://github.com/hwchase17/langchain

* 설명 유튜브 영상: https://youtu.be/KdbPZNdFJU0
---

In [ ]:
#@title 기본 패키지(openai, langchain) 설치
!pip install openai
!pip install langchain

In [ ]:
#@title 기타 패키지 설치 (구글검색, 위키피디아, VectorStore, HuggingFace Embedding)
!pip install google-search-results
!pip install wikipedia
!pip install faiss-cpu # 오픈소스 벡터DB (Facebook, MIT license)
!pip install sentence_transformers # HuggingFace Embedding 사용 위해서 필요
!pip install tiktoken # Summarization 할때 필요

In [ ]:
#@title 0. API 키 설정
import os
#@markdown https://platform.openai.com/account/api-keys
OPENAI_API_KEY = "" #@param {type:"string"}

#@markdown https://huggingface.co/settings/tokens
#@markdown HuggingFace에서 모델 다운로드나 클라우드 모델 사용하기 위해서 필요 (무료)
HUGGINGFACEHUB_API_TOKEN = "" #@param {type:"string"}

#@markdown https://serpapi.com/manage-api-key
#@markdown 구글 검색하기 위해서 필요 (월 100회 무료)
SERPAPI_API_KEY = "" #@param {type:"string"}

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY


In [ ]:
#@title 1. OpenAI LLM (text-davinci-003)

from langchain.llms import OpenAI

llm = OpenAI(model_name='text-davinci-003', temperature=0.9)

llm('1980년대 메탈 음악 5곡 추천해줘.')

"\n\n1. Keep the Faith - Bon Jovi \n2. Ride the Lightning - Metallica \n3. Crazy Train - Ozzy Osbourne \n4. Welcome to the Jungle - Guns N' Roses \n5. Livin' on a Prayer - Bon Jovi"

In [ ]:
#@title 2. ChatOpenAI LLM (gpt-3.5-turbo)

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)
sys = SystemMessage(content="당신은 음악 추천을 해주는 전문 AI입니다.")
msg = HumanMessage(content='1980년대 메탈 음악 5곡 추천해줘.')

aimsg = chat([sys, msg])
aimsg.content

'1. "Master of Puppets" - Metallica\n2. "Crazy Train" - Ozzy Osbourne\n3. "Living After Midnight" - Judas Priest\n4. "Ace of Spades" - Motörhead\n5. "Paranoid" - Black Sabbath'

In [ ]:
#@title 3. Prompt Template & chain

from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["상품"],
    template="{상품} 만드는 회사 이름 추천해줘. 기억에 남는 한글 이름으로",
)

prompt.format(상품="AI 여행 추천 서비스")

'AI 여행 추천 서비스 만드는 회사 이름 추천해줘. 기억에 남는 한글 이름으로'

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=chat, prompt=prompt)

# chain.run("AI 여행 추천 서비스")
chain.run(상품="AI 여행 추천 서비스")

'여행이야기 (Yeo-haeng-i-ya-gi)'

In [ ]:
#@title 4. ChatPromptTemplate & chain

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template="You are a helpful assisstant that tranlates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


chatchain = LLMChain(llm=chat, prompt=chat_prompt)
chatchain.run(input_language="English", output_language="Korean", text="I love programming.")


'저는 프로그래밍을 좋아합니다.'

In [ ]:
#@title 5. Agents and Tools

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

* Tool: 특정 일을 하는 역할. 예) 구글 검색, 디비 조회, Python 실행/계산.
* Agents: 어떤 툴을 어떤 순서로 실행할지 결정하는 역할 (LLM 이용)

In [ ]:
# tools = load_tools(["serpapi", "llm-math"], llm=chat)
tools = load_tools(["wikipedia", "llm-math"], llm=chat)

agent = initialize_agent(tools, llm=chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("페이스북 창업자는 누구인지? 그의 현재(2023년) 나이를 제곱하면?")



> Entering new AgentExecutor chain...
I need to find out who the founder of Facebook is and then square their current age in 2023.
Action: Wikipedia
Action Input: "Founder of Facebook"
Observation: Page: Facebook
Summary: Facebook is an online social media and social networking service owned by American technology giant Meta Platforms. Created in 2004 by Mark Zuckerberg with fellow Harvard College students and roommates Eduardo Saverin, Andrew McCollum, Dustin Moskovitz, and Chris Hughes, its name derives from the face book directories often given to American university students. Membership was initially limited to only Harvard students, gradually expanding to other North American universities and, since 2006, anyone over 13 years old. As of December 2022, Facebook claimed 2.96 billion monthly active users, and ranked third worldwide among the most visited websites. It was the most downloaded mobile app of the 2010s.Facebook can be accessed from devices with Internet connectivity, su

'Mark Zuckerberg is the founder of Facebook and his age in 2023, when squared, is 1521.'

In [ ]:
agent.tools

[WikipediaQueryRun(name='Wikipedia', description='A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, historical events, or other subjects. Input should be a search query.', return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fd3844ffbb0>, api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.9/dist-packages/wikipedia/__init__.py'>, top_k_results=3)),
 Tool(name='Calculator', description='Useful for when you need to answer questions about math.', return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fd3844ffbb0>, func=<bound method Chain.run of LLMMathChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fd3844ffbb0>, verbose=False, llm=ChatOpenAI(verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallba

In [ ]:
print(agent.tools[0].description)
print(agent.tools[1].description)

A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, historical events, or other subjects. Input should be a search query.
Useful for when you need to answer questions about math.


In [ ]:
#@title 6. Memory
from langchain import ConversationChain

conversation = ConversationChain(llm=chat, verbose=True)
conversation.predict(input="인공지능에서 Transformer가 뭐야?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 인공지능에서 Transformer가 뭐야?
AI:

> Finished chain.


'Transformer는 자연어 처리 분야에서 매우 유용한 딥러닝 모델 중 하나입니다. 이 모델은 기계 번역, 질의응답, 요약 등 다양한 자연어 처리 작업에서 사용됩니다. Transformer는 기존의 RNN, LSTM 등의 모델보다 더욱 빠르고 정확한 결과를 제공합니다. 이 모델은 구글에서 개발되었으며, 현재까지도 많은 연구자들이 이를 활용하여 다양한 자연어 처리 연구를 수행하고 있습니다.'

In [ ]:
conversation.predict(input="RNN하고 차이 설명해줘.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 인공지능에서 Transformer가 뭐야?
AI: Transformer는 자연어 처리 분야에서 매우 유용한 딥러닝 모델 중 하나입니다. 이 모델은 기계 번역, 질의응답, 요약 등 다양한 자연어 처리 작업에서 사용됩니다. Transformer는 기존의 RNN, LSTM 등의 모델보다 더욱 빠르고 정확한 결과를 제공합니다. 이 모델은 구글에서 개발되었으며, 현재까지도 많은 연구자들이 이를 활용하여 다양한 자연어 처리 연구를 수행하고 있습니다.
Human: RNN하고 차이 설명해줘.
AI:

> Finished chain.


'RNN은 Recurrent Neural Network의 약자로, 시퀀스 데이터를 처리하는 데 특화된 딥러닝 모델입니다. RNN은 이전 시점의 출력값을 현재 시점의 입력값으로 사용하여 시퀀스 데이터를 처리합니다. 하지만 RNN은 시퀀스가 길어질수록 기울기 소실 문제가 발생하여 성능이 저하됩니다. 반면 Transformer는 Self-Attention 메커니즘을 사용하여 입력 시퀀스의 모든 위치를 동시에 고려할 수 있으며, 이를 통해 기울기 소실 문제를 해결합니다. 또한 Transformer는 병렬 처리가 가능하여 RNN보다 더욱 빠른 속도로 처리할 수 있습니다.'

In [ ]:
conversation.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='인공지능에서 Transformer가 뭐야?', additional_kwargs={}), AIMessage(content='Transformer는 자연어 처리 분야에서 매우 유용한 딥러닝 모델 중 하나입니다. 이 모델은 기계 번역, 질의응답, 요약 등 다양한 자연어 처리 작업에서 사용됩니다. Transformer는 기존의 RNN, LSTM 등의 모델보다 더욱 빠르고 정확한 결과를 제공합니다. 이 모델은 구글에서 개발되었으며, 현재까지도 많은 연구자들이 이를 활용하여 다양한 자연어 처리 연구를 수행하고 있습니다.', additional_kwargs={}), HumanMessage(content='RNN하고 차이 설명해줘.', additional_kwargs={}), AIMessage(content='RNN은 Recurrent Neural Network의 약자로, 시퀀스 데이터를 처리하는 데 특화된 딥러닝 모델입니다. RNN은 이전 시점의 출력값을 현재 시점의 입력값으로 사용하여 시퀀스 데이터를 처리합니다. 하지만 RNN은 시퀀스가 길어질수록 기울기 소실 문제가 발생하여 성능이 저하됩니다. 반면 Transformer는 Self-Attention 메커니즘을 사용하여 입력 시퀀스의 모든 위치를 동시에 고려할 수 있으며, 이를 통해 기울기 소실 문제를 해결합니다. 또한 Transformer는 병렬 처리가 가능하여 RNN보다 더욱 빠른 속도로 처리할 수 있습니다.', additional_kwargs={})]), output_key=None, input_key=None, return_messages=False, human_prefix='Human', ai_prefix='AI', memory_key='history')

* ConversationBufferMemory : 대화 기록(기본)
* ConversationBufferWindowMemory : 마지막 n개의 대화만 기억
* Entity Memory : 개체에 대한 정보를 저장
* Conversation Knowledge Graph Memory: 개체의 triple 저장: (sam, 좋아하는 색, 파랑)
* ConversationSummaryMemory : 대화의 요약본을 저장
* ConversationSummaryBufferMemory : 대화 요약본 + 마지막 n토큰 기억
* ConversationTokenBufferMemory : 마지막 n토큰 기억
* VectorStore-Backed Memory : 벡터DB에 정보 저장

In [ ]:
#@title 7. Document Loaders
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_path="https://ko.wikipedia.org/wiki/NewJeans")

documents = loader.load()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
len(docs)

# 4096 token = 3000 English word

10

In [ ]:
print(docs[1].page_content)

---
* Web Page
* PDF
* Email
* Twitter
* DataFrame (pandas)
* YouTube
* Notion
* Google Drive
* Powerpoint, Word, 
* ...
---

In [ ]:
#@title 8. Summarization
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(chat, chain_type="map_reduce", verbose=True)
chain.run(docs[:3])


* "stuff": LLM 한번에 다 보냄. 길면 오류남
* "map_reduce": 나눠서 요약, 전체 요약본 다시 요약
* "refine": (요약 + 다음 문서) => 요약
* "map_rerank": 점수매겨서 중요한거로 요약

In [ ]:
#@title 9. Embeddings and VectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings()

from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

index = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=embeddings,
    # text_splitter=text_splitter,
    ).from_loaders([loader])

# 파일로 저장
index.vectorstore.save_local("faiss-nj")

* Embedding : LLM에 input(text)를 넣었을때 나오는 특징 벡터 - Neural Network의 특정 메모리를 읽어서 만들고, 보통 512개 이상의 float로 만들어짐.
* 의미상 비슷한 input이 들어갔을때, 나오는 특징 벡터가 유사한 특징을 갖고, 거리상 가깝다.
* OpenAI에서 text를 넣으면 Embedding을 돌려주는 API를 제공한다. (다만 비용 발생)
* 로컬에서 유사한 알고리즘을 돌릴수 있다. (다만 똑같은 벡터는 아니고, openAI 임베딩과는 비교할수 없다)
* HuggingFaceEmbeddings 사용하면 로컬에서 돌릴수 있고, GPU 있다면 가속도 지원함.
* Vector Database는 Embedding 벡터와 텍스트를 저장하는 DB
  * Pinecone 서비스 (유료, 무료 제한적)
  * FAISS (페이스북, 오픈소스, 로컬)
  * Embeddings -> 텍스트를 저장
  * Embeddings로 검색할수 있으며, 거리상 가장 가까운 항목을 가져올수 있음. (거리를 구하는 방법은 여러가지)
  * 유사한 의미를 갖는 텍스트에 대한 Embedding 벡터는 거리상 가깝다.

In [ ]:
index.query("뉴진스의 데뷔곡은?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'뉴진스의 데뷔곡은 "Attention"입니다.'

In [ ]:
index.query("뉴진스의 데뷔 멤버는?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'뉴진스(NewJeans)의 데뷔 멤버는 민지, 하니, 다니엘, 해린, 혜인으로 총 5명입니다.'

In [ ]:
index.query("멤버의 나이는?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'민지는 18세, 하니는 18세, 다니엘은 18세, 해린은 16세, 혜인은 14세입니다. (2022년 기준)'

In [ ]:
index.query("멤버의 나이는? (오늘은 2023년 4월 18일)", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'민지는 2004년 5월 7일생으로 18살, 하니는 2004년 10월 6일생으로 18살, 다니엘은 2005년 4월 11일생으로 18살, 해린은 2006년 5월 15일생으로 16살, 혜인은 2008년 4월 21일생으로 14살입니다. (오늘은 2023년 4월 18일)'

In [ ]:
#@title FAISS 벡터DB 디스크에서 불러오기
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

fdb = FAISS.load_local("faiss-nj", embeddings)
index2 = VectorStoreIndexWrapper(vectorstore=fdb)

In [ ]:
index2.query("뉴진스의 데뷔 멤버는?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'뉴진스(NewJeans)의 데뷔 멤버는 민지, 하니, 다니엘, 해린, 혜인으로 총 5명입니다.'